In [6]:
import rdkit
from rdkit import Chem
from rdkit.Chem import rdmolops

In [7]:
from naglmbis.models import load_charge_model
# load two pre-trained charge models
gas_model = load_charge_model("nagl-gas-charge-dipole-esp-wb-default")
water_model = load_charge_model("nagl-water-charge-dipole-esp-wb-default")


In [8]:
from openff.toolkit.topology import Molecule

# create ethanol
methanol = Molecule.from_smiles("CO")
# predict the charges (in e) and atomic volumes in (bohr ^3)
gas_charges = gas_model.compute_properties(methanol.to_rdkit())["mbis-charges"]
water_charges = water_model.compute_properties(methanol.to_rdkit())["mbis-charges"]
print(gas_charges)
print(water_charges)

tensor([[-0.1104],
        [-0.5162],
        [ 0.0869],
        [ 0.0869],
        [ 0.0869],
        [ 0.3658]], grad_fn=<CatBackward0>)
tensor([[-0.0731],
        [-0.6216],
        [ 0.0891],
        [ 0.0891],
        [ 0.0891],
        [ 0.4274]], grad_fn=<CatBackward0>)


In [9]:
from naglmbis.models.base_model import ComputePartialPolarised

polarised_model = ComputePartialPolarised(
    model_gas=gas_model,
    model_water=water_model,
    alpha = 0.5
)

polarised_model.compute_polarised_charges(methanol.to_rdkit())

tensor([[-0.0918],
        [-0.5689],
        [ 0.0880],
        [ 0.0880],
        [ 0.0880],
        [ 0.3966]], grad_fn=<AddBackward0>)